In [2]:
import nibabel as nib
import os
import matplotlib.pyplot as plt

In [ ]:
import os
import nibabel as nib
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def convert_nii_to_jpg(input_folder, output_folder, mode="timeseries"):
    """
    Convert 4D NIfTI files to JPG images.
    
    Parameters:
    - input_folder: Path to folder containing .nii files
    - output_folder: Path to save JPG files
    - mode: How to handle 4D data:
        - "timeseries": Save one folder per volume with all time points for a selected slice
        - "spatial": Save one folder per volume with all spatial slices for a selected time point
        - "mean": Save one folder per volume with mean image across time for each slice
    """
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get all .nii files in the input folder
    nii_files = [f for f in os.listdir(input_folder) if f.endswith('.nii') or f.endswith('.nii.gz')]
    
    for nii_file in nii_files:
        # Load the NIfTI file
        nii_path = os.path.join(input_folder, nii_file)
        img = nib.load(nii_path)
        data = img.get_fdata()
        
        # Get base filename without extension
        base_name = os.path.splitext(nii_file)[0]
        if base_name.endswith('.nii'):
            base_name = os.path.splitext(base_name)[0]
        
        print(f"Processing {nii_file} with shape {data.shape}")
        
        # Create a specific folder for this file's outputs
        file_output_folder = os.path.join(output_folder, base_name)
        if not os.path.exists(file_output_folder):
            os.makedirs(file_output_folder)
        
        # Handle 4D data
        if len(data.shape) == 4:
            x_dim, y_dim, z_dim, t_dim = data.shape
            
            if mode == "timeseries":
                # Choose a middle slice in the z dimension
                z_middle = z_dim // 2
                
                # Save all time points for the middle slice
                for time_idx in range(t_dim):
                    slice_data = data[:, :, z_middle, time_idx]
                    
                    # Normalize to 0-255 for JPG
                    normalized_data = ((slice_data - slice_data.min()) / 
                                     (slice_data.max() - slice_data.min() + 1e-10) * 255).astype(np.uint8)
                    
                    # Create a PIL image and save as JPG
                    img_pil = Image.fromarray(normalized_data)
                    output_path = os.path.join(file_output_folder, f"timepoint_{time_idx}.jpg")
                    img_pil.save(output_path)
                
                print(f"Converted {nii_file} - {t_dim} time points saved for z-slice {z_middle}")
                
            elif mode == "spatial":
                # Choose a middle time point
                t_middle = t_dim // 2
                
                # Save all spatial slices for the middle time point
                for z_idx in range(z_dim):
                    slice_data = data[:, :, z_idx, t_middle]
                    
                    # Normalize to 0-255 for JPG
                    normalized_data = ((slice_data - slice_data.min()) / 
                                     (slice_data.max() - slice_data.min() + 1e-10) * 255).astype(np.uint8)
                    
                    # Create a PIL image and save as JPG
                    img_pil = Image.fromarray(normalized_data)
                    output_path = os.path.join(file_output_folder, f"slice_{z_idx}.jpg")
                    img_pil.save(output_path)
                
                print(f"Converted {nii_file} - {z_dim} spatial slices saved for time point {t_middle}")
                
            elif mode == "mean":
                # Calculate mean across time dimension
                mean_data = np.mean(data, axis=3)
                
                # Save each spatial slice of the mean data
                for z_idx in range(z_dim):
                    slice_data = mean_data[:, :, z_idx]
                    
                    # Normalize to 0-255 for JPG
                    normalized_data = ((slice_data - slice_data.min()) / 
                                     (slice_data.max() - slice_data.min() + 1e-10) * 255).astype(np.uint8)
                    
                    # Create a PIL image and save as JPG
                    img_pil = Image.fromarray(normalized_data)
                    output_path = os.path.join(file_output_folder, f"mean_slice_{z_idx}.jpg")
                    img_pil.save(output_path)
                
                print(f"Converted {nii_file} - {z_dim} mean slices saved")
                
            elif mode == "montage":
                # Create a montage of slices at a single time point
                t_middle = t_dim // 2  # Choose middle time point
                
                # Create a figure with subplots for each slice
                n_cols = 6  # Number of columns in the montage
                n_rows = (z_dim + n_cols - 1) // n_cols  # Calculate needed rows
                
                plt.figure(figsize=(15, 3 * n_rows))
                
                for z_idx in range(z_dim):
                    plt.subplot(n_rows, n_cols, z_idx + 1)
                    plt.imshow(data[:, :, z_idx, t_middle], cmap='gray')
                    plt.title(f"Slice {z_idx}")
                    plt.axis('off')
                
                plt.tight_layout()
                montage_path = os.path.join(file_output_folder, f"montage_timepoint_{t_middle}.jpg")
                plt.savefig(montage_path)
                plt.close()
                
                print(f"Created montage for {nii_file} at time point {t_middle}")
                
        else:
            print(f"Skipping {nii_file} - expected 4D data but got {len(data.shape)}D")

# Example usage
input_folder = r"E:\Dataset\DX1"  # Your folder path
output_folder = r"E:\Dataset\DX1_JPG"  # Where to save the JPG files

convert_nii_to_jpg(input_folder, output_folder, mode="montage")  # Create slice montages

Processing 29177.nii with shape (64, 80, 33, 180)
Created montage for 29177.nii at time point 90
Processing 29179.nii with shape (64, 80, 33, 180)
Created montage for 29179.nii at time point 90
Processing 29180.nii with shape (64, 80, 33, 180)
Created montage for 29180.nii at time point 90
Processing 29213.nii with shape (64, 80, 33, 180)
Created montage for 29213.nii at time point 90
Processing 29217.nii with shape (64, 80, 33, 172)
Created montage for 29217.nii at time point 86
Processing 29273.nii with shape (96, 96, 47, 128)
Created montage for 29273.nii at time point 64
Processing 29274.nii with shape (96, 96, 47, 124)
Created montage for 29274.nii at time point 62
Processing 29275.nii with shape (96, 96, 47, 128)
Created montage for 29275.nii at time point 64
Processing 29276.nii with shape (96, 96, 47, 128)
Created montage for 29276.nii at time point 64
Processing 29278.nii with shape (96, 96, 47, 128)
Created montage for 29278.nii at time point 64
Processing 29279.nii with shap

In [12]:
import os
import nibabel as nib
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def convert_nii_to_jpg(input_folder, output_folder, mode="slice_by_slice"):
    """
    Convert 4D NIfTI files to JPG images.
    
    Parameters:
    - input_folder: Path to folder containing .nii files
    - output_folder: Path to save JPG files
    - mode: How to handle 4D data:
        - "slice_by_slice": Save each slice as an individual numbered JPG file
        - "timeseries": Save one folder per volume with all time points for a selected slice
        - "spatial": Save one folder per volume with all spatial slices for a selected time point
        - "mean": Save one folder per volume with mean image across time for each slice
        - "montage": Create a montage of all slices at a middle time point
    """
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get all .nii files in the input folder
    nii_files = [f for f in os.listdir(input_folder) if f.endswith('.nii') or f.endswith('.nii.gz')]
    
    for nii_file in nii_files:
        # Load the NIfTI file
        nii_path = os.path.join(input_folder, nii_file)
        img = nib.load(nii_path)
        data = img.get_fdata()
        
        # Get base filename without extension
        base_name = os.path.splitext(nii_file)[0]
        if base_name.endswith('.nii'):
            base_name = os.path.splitext(base_name)[0]
        
        print(f"Processing {nii_file} with shape {data.shape}")
        
        # Create a specific folder for this file's outputs
        file_output_folder = os.path.join(output_folder, base_name)
        if not os.path.exists(file_output_folder):
            os.makedirs(file_output_folder)
        
        # Handle 4D data
        if len(data.shape) == 4:
            x_dim, y_dim, z_dim, t_dim = data.shape
            
            if mode == "slice_by_slice":
                # Choose a middle time point
                t_middle = t_dim // 2
                
                # Save all spatial slices with simple numeric filenames
                for z_idx in range(z_dim):
                    slice_data = data[:, :, z_idx, t_middle]
                    
                    # Normalize to 0-255 for JPG
                    normalized_data = ((slice_data - slice_data.min()) / 
                                     (slice_data.max() - slice_data.min() + 1e-10) * 255).astype(np.uint8)
                    
                    # Create a PIL image and save as JPG with simple numeric filename
                    img_pil = Image.fromarray(normalized_data)
                    output_path = os.path.join(file_output_folder, f"{z_idx}.jpg")
                    img_pil.save(output_path)
                
                print(f"Converted {nii_file} - {z_dim} slices saved as individual JPG files")
                
            elif mode == "timeseries":
                # Choose a middle slice in the z dimension
                z_middle = z_dim // 2
                
                # Save all time points for the middle slice
                for time_idx in range(t_dim):
                    slice_data = data[:, :, z_middle, time_idx]
                    
                    # Normalize to 0-255 for JPG
                    normalized_data = ((slice_data - slice_data.min()) / 
                                     (slice_data.max() - slice_data.min() + 1e-10) * 255).astype(np.uint8)
                    
                    # Create a PIL image and save as JPG
                    img_pil = Image.fromarray(normalized_data)
                    output_path = os.path.join(file_output_folder, f"timepoint_{time_idx}.jpg")
                    img_pil.save(output_path)
                
                print(f"Converted {nii_file} - {t_dim} time points saved for z-slice {z_middle}")
                
            elif mode == "spatial":
                # Choose a middle time point
                t_middle = t_dim // 2
                
                # Save all spatial slices for the middle time point
                for z_idx in range(z_dim):
                    slice_data = data[:, :, z_idx, t_middle]
                    
                    # Normalize to 0-255 for JPG
                    normalized_data = ((slice_data - slice_data.min()) / 
                                     (slice_data.max() - slice_data.min() + 1e-10) * 255).astype(np.uint8)
                    
                    # Create a PIL image and save as JPG
                    img_pil = Image.fromarray(normalized_data)
                    output_path = os.path.join(file_output_folder, f"slice_{z_idx}.jpg")
                    img_pil.save(output_path)
                
                print(f"Converted {nii_file} - {z_dim} spatial slices saved for time point {t_middle}")
                
            elif mode == "mean":
                # Calculate mean across time dimension
                mean_data = np.mean(data, axis=3)
                
                # Save each spatial slice of the mean data
                for z_idx in range(z_dim):
                    slice_data = mean_data[:, :, z_idx]
                    
                    # Normalize to 0-255 for JPG
                    normalized_data = ((slice_data - slice_data.min()) / 
                                     (slice_data.max() - slice_data.min() + 1e-10) * 255).astype(np.uint8)
                    
                    # Create a PIL image and save as JPG
                    img_pil = Image.fromarray(normalized_data)
                    output_path = os.path.join(file_output_folder, f"mean_slice_{z_idx}.jpg")
                    img_pil.save(output_path)
                
                print(f"Converted {nii_file} - {z_dim} mean slices saved")
                
            elif mode == "montage":
                # Create a montage of slices at a single time point
                t_middle = t_dim // 2  # Choose middle time point
                
                # Create a figure with subplots for each slice
                n_cols = 6  # Number of columns in the montage
                n_rows = (z_dim + n_cols - 1) // n_cols  # Calculate needed rows
                
                plt.figure(figsize=(15, 3 * n_rows))
                
                for z_idx in range(z_dim):
                    plt.subplot(n_rows, n_cols, z_idx + 1)
                    plt.imshow(data[:, :, z_idx, t_middle], cmap='gray')
                    plt.title(f"Slice {z_idx}")
                    plt.axis('off')
                
                plt.tight_layout()
                montage_path = os.path.join(file_output_folder, f"montage_timepoint_{t_middle}.jpg")
                plt.savefig(montage_path)
                plt.close()
                
                print(f"Created montage for {nii_file} at time point {t_middle}")
                
        else:
            print(f"Skipping {nii_file} - expected 4D data but got {len(data.shape)}D")

# Example usage
input_folder = r"E:\Dataset\DX2"  # Your folder path
output_folder = r"E:\Dataset\DX2_JPG"  # Where to save the JPG files

# Use the new "slice_by_slice" mode to save slices with simple numeric filenames
convert_nii_to_jpg(input_folder, output_folder, mode="slice_by_slice")

Processing 29295.nii with shape (96, 96, 47, 128)
Converted 29295.nii - 47 slices saved as individual JPG files
Processing 29297.nii with shape (96, 96, 47, 128)
Converted 29297.nii - 47 slices saved as individual JPG files
Processing 29300.nii with shape (96, 96, 47, 128)
Converted 29300.nii - 47 slices saved as individual JPG files
Processing 29301.nii with shape (96, 96, 47, 128)
Converted 29301.nii - 47 slices saved as individual JPG files
Processing 29302.nii with shape (96, 96, 47, 128)
Converted 29302.nii - 47 slices saved as individual JPG files
Processing 29304.nii with shape (96, 96, 47, 128)
Converted 29304.nii - 47 slices saved as individual JPG files
Processing 29306.nii with shape (96, 96, 47, 128)
Converted 29306.nii - 47 slices saved as individual JPG files
Processing 29312.nii with shape (96, 96, 47, 128)
Converted 29312.nii - 47 slices saved as individual JPG files
Processing 29444.nii with shape (96, 96, 47, 156)
Converted 29444.nii - 47 slices saved as individual JP